In [1]:
import requests
import pandas as pd

In [2]:
#variables

entity = 'https://iot.hamburg.de/v1.1/Things?'
rows = '$skip=0&$top=5000&'
#gefiltert nach Fahrradzählstationen
filter = '$filter=((properties%2FownerThing+eq+%27Hamburg+Verkehrsanlagen%27))'
filter_observation = '?$skip=0&$top=3&$orderby=resultTime+desc'

url = entity + rows + filter

In [3]:
response = requests.get(url)
things_data = response.json()

In [4]:
# get a list of datastream_link from every thing
datastreams = []
for thing in things_data['value']:
    thing = thing['Datastreams@iot.navigationLink'] 
    datastreams.append(thing)

In [5]:
## get a list of datastreams from every thing
# all links with last 3 observations
observations = []
for stream in datastreams:
    stream = requests.get(stream)
    data = stream.json()
    value = data['value']
    link = value[0]['Observations@iot.navigationLink']+filter_observation
    observations.append(link)

In [ ]:
final_df = pd.DataFrame() 
result_list = []
resultTime_list = []
coordinates_list = []

for observation in observations:
    request = requests.get(observation)
    data = request.json()
    values = data['value']

    for item in values:
        # result = counts and resulttime = time
        result_list.append(item['result'])
        resultTime_list.append(item['resultTime'])

        # holding coordinates
        link = item['FeatureOfInterest@iot.navigationLink']

        # Request 'FeatureOfInterest@iot.navigationLink'
        request = requests.get(link)
        feature_of_interest = request.json()
        coordinates = feature_of_interest['feature']['geometry']['coordinates']
        coordinates_list.append(coordinates)

df = pd.DataFrame({'result': result_list, 'resultTime': resultTime_list, 'coordinates': coordinates_list})
final_df = pd.concat([final_df, df], ignore_index=True)

In [ ]:
df.head(2)